# 🐼 Pandas - Class 7: Merging, Joining & Concatenation
Welcome to **Class 7** of our Pandas series. Today we’ll learn how to combine multiple DataFrames using different techniques.

## 1. Using `concat()` for Stacking Data
- `pd.concat()` combines DataFrames vertically (rows) or horizontally (columns).
- Use `axis=0` for vertical stacking (default), `axis=1` for horizontal.
- `ignore_index=True` can reset the index after concatenation.

In [1]:
import pandas as pd

# Dataset 1: Product details
df1 = pd.DataFrame({
    "ProductID": [101, 102, 103],
    "ProductName": ["Pen", "Notebook", "Marker"],
    "Price": [10, 50, 30]
})

# Dataset 2: More products (same columns as df1)
df2 = pd.DataFrame({
    "ProductID": [104, 105],
    "ProductName": ["Pencil", "Eraser"],
    "Price": [5, 8]
})

# Dataset 3: Additional info (different columns)
df3 = pd.DataFrame({
    "Stock": [100, 60, 80],
    "Rating": [4.5, 4.7, 4.8]
})

print("Original DataFrames:")
print("\nDF1:\n", df1)
print("\nDF2:\n", df2)
print("\nDF3:\n", df3)

Original DataFrames:

DF1:
    ProductID ProductName  Price
0        101         Pen     10
1        102    Notebook     50
2        103      Marker     30

DF2:
    ProductID ProductName  Price
0        104      Pencil      5
1        105      Eraser      8

DF3:
    Stock  Rating
0    100     4.5
1     60     4.7
2     80     4.8


In [2]:
# 1. Vertical stacking (rows) — combine df1 and df2
vertical = pd.concat([df1, df2], ignore_index=True)
print("\nVertical concatenation of df1 & df2:")
vertical


Vertical concatenation of df1 & df2:


,ProductID,ProductName,Price
0,101,Pen,10
1,102,Notebook,50
2,103,Marker,30
3,104,Pencil,5
4,105,Eraser,8


In [3]:
# 2. Horizontal stacking (columns) — combine df1 and df3
horizontal = pd.concat([df1, df3], axis=1)
print("\nHorizontal concatenation of df1 & df3:")
horizontal


Horizontal concatenation of df1 & df3:


,ProductID,ProductName,Price,Stock,Rating
0,101,Pen,10,100,4.5
1,102,Notebook,50,60,4.7
2,103,Marker,30,80,4.8


In [4]:
# 3. Horizontal stacking with different index alignment
# (e.g., leave default index to see alignment effect)
horizontal_no_reset = pd.concat([df1, df3], axis=1, ignore_index=False)
print("\nHorizontal concat without resetting index:")
horizontal_no_reset


Horizontal concat without resetting index:


,ProductID,ProductName,Price,Stock,Rating
0,101,Pen,10,100,4.5
1,102,Notebook,50,60,4.7
2,103,Marker,30,80,4.8


## 2. Using `merge()` for SQL-Style Joins
- `merge()` is similar to SQL joins.
- Specify `on` or `left_on`/`right_on` to choose key columns.
- Join types: `inner` (default), `left`, `right`, `outer`.

In [5]:
# Combine vertically so we have one product table
products = pd.concat([df1, df2], ignore_index=True)

# Sales data (ProductID is the key for joining)
sales = pd.DataFrame({
    "ProductID": [101, 102, 104, 106],  # 106 doesn't exist in products
    "UnitsSold": [500, 200, 150, 50]
})

print("Products DataFrame:")
print(products)
print("\nSales DataFrame:")
print(sales)

Products DataFrame:
   ProductID ProductName  Price
0        101         Pen     10
1        102    Notebook     50
2        103      Marker     30
3        104      Pencil      5
4        105      Eraser      8

Sales DataFrame:
   ProductID  UnitsSold
0        101        500
1        102        200
2        104        150
3        106         50


In [6]:
# 1. Inner join (only matching ProductID)
inner = pd.merge(products, sales, on="ProductID", how="inner")
print("\nInner Join (only matches):")
inner


Inner Join (only matches):


,ProductID,ProductName,Price,UnitsSold
0,101,Pen,10,500
1,102,Notebook,50,200
2,104,Pencil,5,150


In [7]:
# 2. Left join (all products, add sales where available)
left = pd.merge(products, sales, on="ProductID", how="left")
print("\nLeft Join (all products):")
left


Left Join (all products):


,ProductID,ProductName,Price,UnitsSold
0,101,Pen,10,500.0
1,102,Notebook,50,200.0
2,103,Marker,30,NaN
3,104,Pencil,5,150.0
4,105,Eraser,8,NaN


In [8]:
# 3. Right join (all sales, add product info where available)
right = pd.merge(products, sales, on="ProductID", how="right")
print("\nRight Join (all sales):")
right


Right Join (all sales):


,ProductID,ProductName,Price,UnitsSold
0,101,Pen,10.0,500
1,102,Notebook,50.0,200
2,104,Pencil,5.0,150
3,106,NaN,NaN,50


In [9]:
# 4. Outer join (everything, even unmatched)
outer = pd.merge(products, sales, on="ProductID", how="outer")
print("\nOuter Join (all data):")
outer


Outer Join (all data):


,ProductID,ProductName,Price,UnitsSold
0,101,Pen,10.0,500.0
1,102,Notebook,50.0,200.0
2,103,Marker,30.0,NaN
3,104,Pencil,5.0,150.0
4,105,Eraser,8.0,NaN
5,106,NaN,NaN,50.0


## 3. Using `join()` with Index Alignment
- `join()` joins DataFrames on their index or on a key column.
- By default, joins on index; can specify `on='col'`.
- Useful for adding columns to an existing DataFrame.

In [12]:
# Combine products into one table
products = pd.concat([df1, df2], ignore_index=True)

# Stock info (different DataFrame, indexed by ProductID)
stock = pd.DataFrame({
    "Stock": [100, 60, 80, 120, 150],
}, index=[101, 102, 103, 104, 105])  # ProductID as index

print("Products DataFrame:")
print(products)
print("\nStock DataFrame (indexed by ProductID):")
print(stock)

Products DataFrame:
   ProductID ProductName  Price
0        101         Pen     10
1        102    Notebook     50
2        103      Marker     30
3        104      Pencil      5
4        105      Eraser      8

Stock DataFrame (indexed by ProductID):
     Stock
101    100
102     60
103     80
104    120
105    150


In [13]:
# 1. Join stock info to products by aligning on ProductID
# First set ProductID as index in products
p_indexed = products.set_index("ProductID")
joined = p_indexed.join(stock)
print("\nJoined DataFrame (products + stock):")
joined


Joined DataFrame (products + stock):


,ProductName,Price,Stock
ProductID,,,
101,Pen,10,100
102,Notebook,50,60
103,Marker,30,80
104,Pencil,5,120
105,Eraser,8,150


In [14]:
# 2. Join using `on` if you don’t want to reset the index
joined_on = products.join(stock, on="ProductID")
print("\nJoined using `on='ProductID'`:")
joined_on


Joined using `on='ProductID'`:


,ProductID,ProductName,Price,Stock
0,101,Pen,10,100
1,102,Notebook,50,60
2,103,Marker,30,80
3,104,Pencil,5,120
4,105,Eraser,8,150


## 4. Practical Examples
- Combine sales and revenue data to see performance.
- Merge students with marks to link records.
- Practice with realistic datasets to understand alignment and missing data handling.

In [15]:
import pandas as pd

# Sales data
sales = pd.DataFrame({
    "ProductID": [201, 202, 203, 204],
    "UnitsSold": [500, 300, 400, 250]
})

# Revenue data (missing ProductID 204, extra 205)
revenue = pd.DataFrame({
    "ProductID": [201, 202, 203, 205],
    "Revenue": [7500, 4500, 6800, 1200]
})

print("Sales Data:")
print(sales)
print("\nRevenue Data:")
print(revenue)

# Merge sales and revenue to see performance
merged_perf = pd.merge(sales, revenue, on="ProductID", how="outer")
print("\nMerged Sales & Revenue (outer join):")
merged_perf


Sales Data:
   ProductID  UnitsSold
0        201        500
1        202        300
2        203        400
3        204        250

Revenue Data:
   ProductID  Revenue
0        201     7500
1        202     4500
2        203     6800
3        205     1200

Merged Sales & Revenue (outer join):


,ProductID,UnitsSold,Revenue
0,201,500.0,7500.0
1,202,300.0,4500.0
2,203,400.0,6800.0
3,204,250.0,NaN
4,205,NaN,1200.0


## Mini Practice
1. Create three DataFrames: Products, Sales, and Revenue.
2. Concatenate them vertically and horizontally.
3. Merge Sales and Revenue using different join types.
4. Join Products with Sales based on index or a key.
5. Analyze missing values after joins.

In [16]:
import pandas as pd

# 1. Create three DataFrames: Products, Sales, and Revenue
products = pd.DataFrame({
    "ProductID": [101, 102, 103, 104],
    "ProductName": ["Pen", "Notebook", "Pencil", "Eraser"]
})

sales = pd.DataFrame({
    "ProductID": [101, 102, 103, 105],
    "UnitsSold": [200, 150, 300, 50]
})

revenue = pd.DataFrame({
    "ProductID": [101, 102, 104, 105],
    "Revenue": [1000, 2500, 800, 200]
})

print("Products DataFrame:")
print(products)
print("\nSales DataFrame:")
print(sales)
print("\nRevenue DataFrame:")
print(revenue)

Products DataFrame:
   ProductID ProductName
0        101         Pen
1        102    Notebook
2        103      Pencil
3        104      Eraser

Sales DataFrame:
   ProductID  UnitsSold
0        101        200
1        102        150
2        103        300
3        105         50

Revenue DataFrame:
   ProductID  Revenue
0        101     1000
1        102     2500
2        104      800
3        105      200


In [18]:
# 2. Concatenate them vertically and horizontally
# Vertical stacking
vertical = pd.concat([products, sales, revenue], ignore_index=True)
print("\nVertical concatenation (all three):")
vertical


Vertical concatenation (all three):


,ProductID,ProductName,UnitsSold,Revenue
0,101,Pen,NaN,NaN
1,102,Notebook,NaN,NaN
2,103,Pencil,NaN,NaN
3,104,Eraser,NaN,NaN
4,101,NaN,200.0,NaN
5,102,NaN,150.0,NaN
6,103,NaN,300.0,NaN
7,105,NaN,50.0,NaN
8,101,NaN,NaN,1000.0
9,102,NaN,NaN,2500.0


In [19]:
# Horizontal stacking (aligns by index)
horizontal = pd.concat([products, sales, revenue], axis=1)
print("\nHorizontal concatenation (side-by-side):")
horizontal


Horizontal concatenation (side-by-side):


,ProductID,ProductName,ProductID,UnitsSold,ProductID,Revenue
0,101,Pen,101,200,101,1000
1,102,Notebook,102,150,102,2500
2,103,Pencil,103,300,104,800
3,104,Eraser,105,50,105,200


In [20]:
# 3. Merge Sales and Revenue using different join types
inner_merge = pd.merge(sales, revenue, on="ProductID", how="inner")
print("\nInner Join (Sales & Revenue):")
inner_merge


Inner Join (Sales & Revenue):


,ProductID,UnitsSold,Revenue
0,101,200,1000
1,102,150,2500
2,105,50,200


In [21]:
left_merge = pd.merge(sales, revenue, on="ProductID", how="left")
print("\nLeft Join (Sales with Revenue):")
left_merge


Left Join (Sales with Revenue):


,ProductID,UnitsSold,Revenue
0,101,200,1000.0
1,102,150,2500.0
2,103,300,NaN
3,105,50,200.0


In [22]:
right_merge = pd.merge(sales, revenue, on="ProductID", how="right")
print("\nRight Join (Revenue with Sales):")
right_merge


Right Join (Revenue with Sales):


,ProductID,UnitsSold,Revenue
0,101,200.0,1000
1,102,150.0,2500
2,104,NaN,800
3,105,50.0,200


In [23]:
outer_merge = pd.merge(sales, revenue, on="ProductID", how="outer")
print("\nOuter Join (all ProductIDs):")
outer_merge


Outer Join (all ProductIDs):


,ProductID,UnitsSold,Revenue
0,101,200.0,1000.0
1,102,150.0,2500.0
2,103,300.0,NaN
3,104,NaN,800.0
4,105,50.0,200.0


In [33]:
# 4. Join Products with Sales based on key
joined = products.merge(sales, on="ProductID", how="left")
print("\nProducts joined with Sales (left join):")
joined


Products joined with Sales (left join):


,ProductID,ProductName,UnitsSold
0,101,Pen,200.0
1,102,Notebook,150.0
2,103,Pencil,300.0
3,104,Eraser,NaN


In [38]:
# 5. Analyze missing values after joins
print("\nMissing values summary (Products + Sales):")
joined.isna().sum()


Missing values summary (Products + Sales):


,0
ProductID,0
ProductName,0
UnitsSold,1


In [30]:
print("\nInfo about joined DataFrame:")
joined.info()


Info about joined DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    4 non-null      int64  
 1   ProductName  4 non-null      object 
 2   UnitsSold    3 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 228.0+ bytes


In [31]:
print("\nDescribe numeric columns (joined DataFrame):")
joined.describe()


Describe numeric columns (joined DataFrame):


,ProductID,UnitsSold
count,4.000000,3.000000
mean,102.500000,216.666667
std,1.290994,76.376262
min,101.000000,150.000000
25%,101.750000,175.000000
50%,102.500000,200.000000
75%,103.250000,250.000000
max,104.000000,300.000000


---
## Summary
- Used `concat()` for stacking data vertically and horizontally.
- Performed SQL-style joins with `merge()`.
- Applied `join()` with index alignment.
- Explored practical use-cases like sales vs revenue, students vs marks.